In [4]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 65.7 MB/s eta 0:00:00:00:0100:01


In [5]:
import json
import spacy
from sentence_transformers import SentenceTransformer
from sklearn.neighbors import NearestNeighbors
import faiss
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, average_precision_score
import time

# Load pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')
nlp = spacy.load('en_core_web_sm')

# Example documents with JSON structure
documents_json = """
[
    {"acl_id": "1", "title": "Example title 1", "abstract": "Example abstract 1", "full_text": "Sentence 1 of doc 1. Sentence 2 of doc 1."},
    {"acl_id": "2", "title": "Example title 2", "abstract": "Example abstract 2", "full_text": "Sentence 1 of doc 2. Sentence 2 of doc 2."},
    {"acl_id": "3", "title": "Example title 3", "abstract": "Example abstract 3", "full_text": "Sentence 1 of doc 3. Sentence 2 of doc 3."},
    {"acl_id": "4", "title": "Example title 4", "abstract": "Example abstract 4", "full_text": "Sentence 1 of doc 4. Sentence 2 of doc 4."}
]
"""

# Load documents from JSON
documents = json.loads(documents_json)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
# Ground truth document set
ground_truth_json = """
[
    {"q": "Sentence 1 of doc 1", "acl_ids": ["1"], "r": "Response 1"},
    {"q": "Sentence 2 of doc 2", "acl_ids": ["2"], "r": "Response 2"},
    {"q": "Sentence 1 of doc 3", "acl_ids": ["3"], "r": "Response 3"}
]
"""

ground_truth_docs = json.loads(ground_truth_json)

In [10]:
import torch
from transformers import BertTokenizer, BertModel
from sentence_transformers import SentenceTransformer
import datasets

In [11]:
# R_NUMBER_SEED = 1234567 # Replace this with your own student number
R_NUMBER_SEED = 928036 # my student number 
DOCS_TO_ADD = 1000
query_documents = datasets.load_dataset("parquet", data_files="./acl_anthology_queries.parquet")["train"]
all_documents = datasets.load_dataset("parquet", data_files="./acl_anthology_full.parquet")["train"]
# Shuffle with seed and take only n docs
shuffled_documents = all_documents.shuffle(seed=R_NUMBER_SEED)
random_documents = shuffled_documents.select(range(DOCS_TO_ADD))
# Concatenate relevant documents with random sample and shuffle again
anthology_sample = datasets.concatenate_datasets([query_documents, random_documents]).shuffle(seed=R_NUMBER_SEED)
# Export to Parquet to avoid downloading full anthology
anthology_sample.to_parquet("./anthology_sample.parquet")

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

27259830

In [18]:
import json
queries = json.load(open("./acl_anthology_queries.json", "r"))
ground_truth_docs = queries["queries"]
documents = anthology_sample

In [19]:
ground_truth_docs[50]

{'q': 'What is the name of the research initiative creating resources for African languages?',
 'r': ['2023.acl-long.796', '2023.acl-long.609', '2023.ijcnlp-main.10'],
 'a': 'Masakhane. (Masakha is also fine.)'}

In [22]:
# Split full_text into sentences and create a mapping to document ids
all_sentences = []
sentence_to_doc_map = []

count = 0
for doc in documents:
    if count % 10 == 0:
        print("doc:",count)
    full_text = ""
    if doc["title"] is not None: full_text = f'{doc["title"]}.' 
    if doc["abstract"] is not None: full_text = f'{full_text} {doc["abstract"]}.' 
    if doc["full_text"] is not None: full_text = f'{full_text} {doc["full_text"]}.' 
    spacy_doc = nlp(full_text)
    sentences = [sent.text.strip() for sent in spacy_doc.sents]
    for sentence in sentences:
        all_sentences.append(sentence)
        sentence_to_doc_map.append(doc["acl_id"])
    count +=1

doc: 1
doc: 2
doc: 3
doc: 4
doc: 5
doc: 6
doc: 7
doc: 8
doc: 9
doc: 11
doc: 12
doc: 13
doc: 14
doc: 15
doc: 16
doc: 17
doc: 18
doc: 19
doc: 21
doc: 22
doc: 23
doc: 24
doc: 25
doc: 26
doc: 27
doc: 28


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f4eb048d4b0>>
Traceback (most recent call last):
  File "/home/marko/virt/py310/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


doc: 29
doc: 31
doc: 32
doc: 33
doc: 34
doc: 35
doc: 36
doc: 37
doc: 38
doc: 39
doc: 41
doc: 42
doc: 43
doc: 44
doc: 45
doc: 46
doc: 47
doc: 48
doc: 49
doc: 51
doc: 52
doc: 53
doc: 54
doc: 55
doc: 56
doc: 57
doc: 58
doc: 59
doc: 61
doc: 62
doc: 63
doc: 64
doc: 65
doc: 66
doc: 67
doc: 68
doc: 69
doc: 71
doc: 72
doc: 73
doc: 74
doc: 75
doc: 76
doc: 77
doc: 78
doc: 79
doc: 81
doc: 82
doc: 83
doc: 84
doc: 85
doc: 86
doc: 87
doc: 88
doc: 89
doc: 91
doc: 92
doc: 93
doc: 94
doc: 95
doc: 96
doc: 97
doc: 98
doc: 99
doc: 101
doc: 102
doc: 103
doc: 104
doc: 105
doc: 106
doc: 107
doc: 108
doc: 109
doc: 111
doc: 112
doc: 113
doc: 114
doc: 115
doc: 116
doc: 117
doc: 118
doc: 119
doc: 121
doc: 122
doc: 123
doc: 124
doc: 125
doc: 126
doc: 127
doc: 128
doc: 129
doc: 131
doc: 132
doc: 133
doc: 134
doc: 135
doc: 136
doc: 137
doc: 138
doc: 139
doc: 141
doc: 142
doc: 143
doc: 144
doc: 145
doc: 146
doc: 147
doc: 148
doc: 149
doc: 151
doc: 152
doc: 153
doc: 154
doc: 155
doc: 156
doc: 157
doc: 158
doc: 159
do

KeyboardInterrupt: 

In [23]:
import nltk
nltk.download('punkt')

from nltk.tokenize import sent_tokenize

start_time = time.time()
def preprocess_documents_nltk(documents):
    all_sentences = []
    all_sentences_to_doc_map = []
    count = 1
    for doc in documents:
        if count % 10 == 0:
            print("doc:",count)
        # print(doc["acl_id"])
        # full_text = f"{doc.get('full_text')}"
        full_text = ""
        if doc["title"] is not None: full_text = f'{doc["title"]}.' 
        if doc["abstract"] is not None: full_text = f'{full_text} {doc["abstract"]}.' 
        if doc["full_text"] is not None: full_text = f'{full_text} {doc["full_text"]}.' 
        sentences = sent_tokenize(full_text)
        for sentence in sentences:
            all_sentences.append(sentence)
            all_sentences_to_doc_map.append(doc["acl_id"])
        count += 1
    return all_sentences,all_sentences_to_doc_map
all_sentences, sentence_to_doc_map = preprocess_documents_nltk(documents)

[nltk_data] Downloading package punkt to /home/marko/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


doc: 10
doc: 20
doc: 30
doc: 40
doc: 50
doc: 60
doc: 70
doc: 80
doc: 90
doc: 100
doc: 110
doc: 120
doc: 130
doc: 140
doc: 150
doc: 160
doc: 170
doc: 180
doc: 190
doc: 200
doc: 210
doc: 220
doc: 230
doc: 240
doc: 250
doc: 260
doc: 270
doc: 280
doc: 290
doc: 300
doc: 310
doc: 320
doc: 330
doc: 340
doc: 350
doc: 360
doc: 370
doc: 380
doc: 390
doc: 400
doc: 410
doc: 420
doc: 430
doc: 440
doc: 450
doc: 460
doc: 470
doc: 480
doc: 490
doc: 500
doc: 510
doc: 520
doc: 530
doc: 540
doc: 550
doc: 560
doc: 570
doc: 580
doc: 590
doc: 600
doc: 610
doc: 620
doc: 630
doc: 640
doc: 650
doc: 660
doc: 670
doc: 680
doc: 690
doc: 700
doc: 710
doc: 720
doc: 730
doc: 740
doc: 750
doc: 760
doc: 770
doc: 780
doc: 790
doc: 800
doc: 810
doc: 820
doc: 830
doc: 840
doc: 850
doc: 860
doc: 870
doc: 880
doc: 890
doc: 900
doc: 910
doc: 920
doc: 930
doc: 940
doc: 950
doc: 960
doc: 970
doc: 980
doc: 990
doc: 1000
doc: 1010
doc: 1020
doc: 1030
doc: 1040
doc: 1050
doc: 1060
doc: 1070
doc: 1080
doc: 1090
doc: 1100
doc: 111

In [24]:
# Create embeddings
embeddings = model.encode(all_sentences)
print("Embeddings created.")

Embeddings created.


In [26]:
# Initialize KNN
knn = NearestNeighbors(n_neighbors=2, metric='cosine')

# Fit the model
knn.fit(embeddings)

# Example queries from ground truth
queries = [doc["q"] for doc in ground_truth_docs]
query_embeddings = model.encode(queries)

In [27]:
# Extract ground truth document IDs
ground_truth = [doc["r"][0] for doc in ground_truth_docs]  # Assuming single correct document per query

In [28]:
# Perform KNN search
knn_indices_list = []
knn_distances_list = []
for query_embedding in query_embeddings:
    distances, indices = knn.kneighbors([query_embedding])
    knn_indices_list.append(indices[0].tolist())
    knn_distances_list.append(distances[0].tolist())

In [29]:
# Initialize FAISS index
embedding_dim = embeddings.shape[1]
faiss_index = faiss.IndexFlatL2(embedding_dim)

# Add embeddings to FAISS index
faiss_index.add(embeddings)
print("FAISS index built.")

# Perform FAISS search
faiss_indices_list = []
faiss_distances_list = []
for query_embedding in query_embeddings:
    distances, indices = faiss_index.search(np.array([query_embedding]), 2)
    faiss_indices_list.append(indices[0].tolist())
    faiss_distances_list.append(distances[0].tolist())

# Convert FAISS indices to document IDs
faiss_predictions = [[sentence_to_doc_map[idx] for idx in indices] for indices in faiss_indices_list]

FAISS index built.


In [31]:
# Convert KNN indices to document IDs
knn_predictions = [[sentence_to_doc_map[idx] for idx in indices] for indices in knn_indices_list]

# Calculate metrics
def calculate_metrics(ground_truth, predictions):
    flattened_ground_truth = [gt for gt in ground_truth for _ in range(2)]
    flattened_predictions = [pred for sublist in predictions for pred in sublist]
    precision = precision_score(flattened_ground_truth, flattened_predictions, average='macro')
    recall = recall_score(flattened_ground_truth, flattened_predictions, average='macro')
    f1 = f1_score(flattened_ground_truth, flattened_predictions, average='macro')
    micro_precision = precision_score(flattened_ground_truth, flattened_predictions, average='micro')
    micro_recall = recall_score(flattened_ground_truth, flattened_predictions, average='micro')
    micro_f1 = f1_score(flattened_ground_truth, flattened_predictions, average='micro')
    map_score = average_precision_score(
        [1 if gt == pred else 0 for gt, pred in zip(flattened_ground_truth, flattened_predictions)], 
        [1] * len(flattened_predictions)
    )
    return precision, recall, f1, micro_precision, micro_recall, micro_f1, map_score

In [33]:
knn_metrics = calculate_metrics(ground_truth, knn_predictions)
faiss_metrics = calculate_metrics(ground_truth, faiss_predictions)

/home/marko/virt/py310/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/marko/virt/py310/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/marko/virt/py310/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/marko/virt/py31

In [34]:
# Print results
print("KNN Results:")
for query, indices, distances in zip(queries, knn_indices_list, knn_distances_list):
    print(f"Query: {query}")
    for idx, dist in zip(indices, distances):
        print(f"Document ID: {sentence_to_doc_map[idx]}, Sentence: {all_sentences[idx]}, Distance: {dist}")

print(f"KNN Metrics - Macro Precision: {knn_metrics[0]}, Macro Recall: {knn_metrics[1]}, Macro F1-Score: {knn_metrics[2]}, Micro Precision: {knn_metrics[3]}, Micro Recall: {knn_metrics[4]}, Micro F1-Score: {knn_metrics[5]}, mAP: {knn_metrics[6]}")

print("FAISS Results:")
for query, indices, distances in zip(queries, faiss_indices_list, faiss_distances_list):
    print(f"Query: {query}")
    for idx, dist in zip(indices, distances):
        print(f"Document ID: {sentence_to_doc_map[idx]}, Sentence: {all_sentences[idx]}, Distance: {dist}")

print(f"FAISS Metrics - Macro Precision: {faiss_metrics[0]}, Macro Recall: {faiss_metrics[1]}, Macro F1-Score: {faiss_metrics[2]}, Micro Precision: {faiss_metrics[3]}, Micro Recall: {faiss_metrics[4]}, Micro F1-Score: {faiss_metrics[5]}, mAP: {faiss_metrics[6]}")

KNN Results:
Query: Which versions of the Morfessor tokenizer have been proposed in the literature?
Document ID: 2020.lrec-1.486, Sentence: While Morfessor 2.0 only implements force splitting certain characters to single-character morphs, i.e., Distance: 0.35286855697631836
Document ID: E14-2006, Sentence: A number of Morfessor variants have been developed later, including Morfessor Categories-MAP (Creutz and Lagus, 2005a) and Allomorfessor (Virpioja et al., 2010) ., Distance: 0.3605763912200928
Query: What is Dynamic Programming Encoding?
Document ID: P97-1038, Sentence: As Chen (1993) points out, dynamic programming is particularly susceptible to deletions occurring in one of the two languages., Distance: 0.41305971145629883
Document ID: 2020.cl-2.5, Sentence: This analysis provides important insights about the process of encoding and its applicability in other tasks., Distance: 0.42070674896240234
Query: How many variants of the byte-pair encoding subword tokenizer have there been p

In [42]:
print(f"FAISS Metrics -\n Macro Precision: {faiss_metrics[0]},\n Macro Recall: {faiss_metrics[1]},\n Macro F1-Score: {faiss_metrics[2]},\n Micro Precision: {faiss_metrics[3]},\n Micro Recall: {faiss_metrics[4]},\n Micro F1-Score: {faiss_metrics[5]},\n mAP: {faiss_metrics[6]}")

FAISS Metrics -
 Macro Precision: 0.228125,
 Macro Recall: 0.19270833333333331,
 Macro F1-Score: 0.19968750000000002,
 Micro Precision: 0.336734693877551,
 Micro Recall: 0.336734693877551,
 Micro F1-Score: 0.336734693877551,
 mAP: 0.336734693877551


In [43]:
print(f"KNN Metrics -\n Macro Precision: {knn_metrics[0]},\n Macro Recall: {knn_metrics[1]},\n Macro F1-Score: {knn_metrics[2]},\n Micro Precision: {knn_metrics[3]},\n Micro Recall: {knn_metrics[4]},\n Micro F1-Score: {knn_metrics[5]},\n mAP: {knn_metrics[6]}")


KNN Metrics -
 Macro Precision: 0.228125,
 Macro Recall: 0.19270833333333331,
 Macro F1-Score: 0.19968750000000002,
 Micro Precision: 0.336734693877551,
 Micro Recall: 0.336734693877551,
 Micro F1-Score: 0.336734693877551,
 mAP: 0.336734693877551


In [44]:
# Calculate metrics
def calculate_metrics(ground_truth, predictions):
    flattened_ground_truth = [gt for sublist in ground_truth for gt in sublist]
    flattened_predictions = [pred for sublist in predictions for pred in sublist]
    
    # Convert ground truth to binary relevance
    relevance = [1 if pred in gt else 0 for pred, gt in zip(flattened_predictions, ground_truth * len(predictions[0]))]
    
    precision = precision_score(flattened_ground_truth, flattened_predictions, average='macro')
    recall = recall_score(flattened_ground_truth, flattened_predictions, average='macro')
    f1 = f1_score(flattened_ground_truth, flattened_predictions, average='macro')
    micro_precision = precision_score(flattened_ground_truth, flattened_predictions, average='micro')
    micro_recall = recall_score(flattened_ground_truth, flattened_predictions, average='micro')
    micro_f1 = f1_score(flattened_ground_truth, flattened_predictions, average='micro')
    map_score = average_precision_score(relevance, [1] * len(relevance))
    return precision, recall, f1, micro_precision, micro_recall, micro_f1, map_score

knn_metrics = calculate_metrics(ground_truth, knn_predictions)
faiss_metrics = calculate_metrics(ground_truth, faiss_predictions)

# Print results
print("KNN Results:")
for query, indices, distances in zip(queries, knn_indices_list, knn_distances_list):
    print(f"Query: {query}")
    for idx, dist in zip(indices, distances):
        print(f"Document ID: {sentence_to_doc_map[idx]}, Sentence: {all_sentences[idx]}, Distance: {dist}")

print(f"KNN Metrics - Macro Precision: {knn_metrics[0]}, Macro Recall: {knn_metrics[1]}, Macro F1-Score: {knn_metrics[2]}, Micro Precision: {knn_metrics[3]}, Micro Recall: {knn_metrics[4]}, Micro F1-Score: {knn_metrics[5]}, mAP: {knn_metrics[6]}")

print("FAISS Results:")
for query, indices, distances in zip(queries, faiss_indices_list, faiss_distances_list):
    print(f"Query: {query}")
    for idx, dist in zip(indices, distances):
        print(f"Document ID: {sentence_to_doc_map[idx]}, Sentence: {all_sentences[idx]}, Distance: {dist}")

print(f"FAISS Metrics - Macro Precision: {faiss_metrics[0]}, Macro Recall: {faiss_metrics[1]}, Macro F1-Score: {faiss_metrics[2]}, Micro Precision: {faiss_metrics[3]}, Micro Recall: {faiss_metrics[4]}, Micro F1-Score: {faiss_metrics[5]}, mAP: {faiss_metrics[6]}")

ValueError: Found input variables with inconsistent numbers of samples: [1277, 196]